In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_path='/content/drive/MyDrive/Datasets/paras_final1.csv'

In [ ]:
!pip3 install transformers datasets -q

In [ ]:
import datasets
datasets.__version__

'2.9.0'

In [ ]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration,AutoTokenizer,TrainingArguments,Trainer

In [ ]:
from numpy.random.mtrand import RandomState
from torch.utils.data.dataset import random_split
dataset=load_dataset("csv",data_files=csv_path,split="train")
dataset=dataset.train_test_split(test_size=0.112, seed=42, shuffle=True)
train_dataset=dataset['train']
test_dataset=dataset['test']
# from sklearn.model_selection import train_test_split

# train, test=train_test_split(dataset, test_size=0.1, random_state=2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2ea72b387f48e9b7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
train_dataset

Dataset({
    features: ['ERRONEOUS_SENTENCE', 'CORRECT_SENTENCE'],
    num_rows: 224440
})

In [ ]:
test_dataset 

Dataset({
    features: ['ERRONEOUS_SENTENCE', 'CORRECT_SENTENCE'],
    num_rows: 28308
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('t5-base', )
def tokenize(batch):
    tokenized_input = tokenizer(batch['ERRONEOUS_SENTENCE'], padding='max_length', truncation=True, max_length=256)
    tokenized_label = tokenizer(batch['CORRECT_SENTENCE'], padding='max_length', truncation=True, max_length=256)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=256)
val_dataset = test_dataset.map(tokenize, batched=True, batch_size=256)

train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalt

  0%|          | 0/877 [00:00<?, ?ba/s]

  0%|          | 0/111 [00:00<?, ?ba/s]

In [ ]:
# train_dataset.save_to_disk('/content/drive/MyDrive/tokenized/train/')
# val_dataset.save_to_disk('/content/drive/MyDrive/tokenized/test/')
# from datasets import load_from_disk
# train=load_from_disk('/content/drive/MyDrive/tokenized/train/')
# test=load_from_disk('/content/drive/MyDrive/tokenized/test/')

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback

model = T5ForConditionalGeneration.from_pretrained('t5-base')

output_dir = '/content/drive/MyDrive/model_saving'
c_point=     '/content/drive/MyDrive/model_saving/Model Checkpoints'
training_args = TrainingArguments(
    output_dir=c_point,
    num_train_epochs=3,
    # weight_decay=0.01, #newlyadded weightdecay                     *
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.0001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=10000, # How often to save a checkpoint
    save_total_limit=10, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    # run_name='run_name', # Wandb run name
    logging_steps=1000, # How often to log loss to wandb
    eval_steps=10000, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False, # Best model is the one with the lowest loss, not highest.
    #push_to_hub=True,
    #push_to_hub_model_id="finetune-t5-for-grammar-correction"
    # fp16=True #newlyadded
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

In [ ]:
import datetime
import pytz

In [ ]:
#for dynamic variable
UTC=pytz.utc
IST=pytz.timezone('Asia/Kolkata')
dt=datetime.datetime.now(IST)
dt2=dt.strftime("%d-%m-%y")
dt3=dt.strftime("%H:%M:%S")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)
trainer.train('/content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-10000')

Loading model from /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-10000.
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: CORRECT_SENTENCE, ERRONEOUS_SENTENCE. If CORRECT_SENTENCE, ERRONEOUS_SENTENCE are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 224440
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 84165
  Number of trainable parameters = 222903552
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 10000
  Will skip the first 0 epochs then the first 10000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` fl

  0%|          | 0/10000 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20000,0.014500,0.011734
30000,0.010800,0.010951
40000,0.009800,0.009903
50000,0.009700,0.009280
60000,0.008000,0.009091
70000,0.008200,0.008700
80000,0.007300,0.008539


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: CORRECT_SENTENCE, ERRONEOUS_SENTENCE. If CORRECT_SENTENCE, ERRONEOUS_SENTENCE are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 28308
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-20000
Configuration saved in /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-20000/config.json
Configuration saved in /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-20000/generation_config.json
Model weights saved in /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-20000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/model_saving/Model Checkpoints/checkpoint-20000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/mode

TrainOutput(global_step=84165, training_loss=0.00883658908872775, metrics={'train_runtime': 21360.0372, 'train_samples_per_second': 31.522, 'train_steps_per_second': 3.94, 'total_flos': 2.050117634359296e+17, 'train_loss': 0.00883658908872775, 'epoch': 3.0})

In [ ]:
import datetime
import pytz

In [ ]:
#for dynamic variable
UTC=pytz.utc
IST=pytz.timezone('Asia/Kolkata')
dt=datetime.datetime.now(IST)
dt2=dt.strftime("%d-%m-%y")
dt3=dt.strftime("%H:%M:%S")

In [ ]:
trainer.save_model(output_dir + f'/paras_model-{dt2}-{dt3}')

Saving model checkpoint to /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02
Configuration saved in /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02/config.json
Configuration saved in /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02/generation_config.json
Model weights saved in /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/model_saving/paras_model-02-02-23-23:12:02/special_tokens_map.json


In [ ]:
import pandas as pd
# df=pd.read_csv("/content/drive/MyDrive/Datasets/Merged mastercopy.csv")
df=pd.read_excel('/content/drive/MyDrive/Datasets/Grammar rues.xlsx')

length=len(df)

In [ ]:
# from typing_extensions import Text
%%time
# input_text = input()
df.dropna(inplace=True)
device="cuda" if torch.cuda.is_available() else "cpu"
predictions=[]
judgement=[]
for i in range(length):
  original=df['Original'][i]
  final=df['Final'][i]
# for i in sent:
  with torch.no_grad():
    tokenized_text = tokenizer(original, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=256,
        num_beams=4,
        temperature=0.3,
        repetition_penalty=0.7, 
        length_penalty=0.6,
        early_stopping=True,
        no_repeat_ngram_size=3,
        top_k=50,
        top_p=0.7
    )
  pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  print(pred)
#   predictions.append(pred)
#   # print(input_text)
#   print (i)
#   print("Original    : ",original)
#   print("\nPrediction  : ",pred)
  

#   # print(original, " : ",pred)
#   print("\nfinal       : ",final)
#   if pred == final:
#     judgement.append('correct')
#     print('\ncorrect \n')
#   # elif pred == original and pred == final:
#   #   judgement.append('correct')
#   #   print('correct \n')
#   elif pred == original and pred != final:
#     judgement.append('no change')
#     print('\nno change \n')
#   elif pred != original and pred != final:
#     judgement.append('check')
#     print('\ncheck \n')
  

#   try:
#     print(f"-Total Correct- = {judgement.count('correct')}\n")
#   except:
#     print("-Total Correct- = 0 \n")
#   try:
#     print(f"---No Changes-- = {judgement.count('no change')}\n")
#   except:
#     print("---No changes-- = 0 \n")
#   try:
#     print(f"-----Check----- = {judgement.count('check')}\n")
#   except:
#     print("-----Check----- = 0 \n")
#   try:
#     t_accuracy=((judgement.count('correct'))/length)*100
#     print(f"---------Accuracy--------- = {t_accuracy}% \n")
#   except:
#     print("---------Accuracy--------- = 0% \n")
#   try:
#     new_decision_percentage=((judgement.count('check'))/length)*100
#     print(f"-new decision percentage-- = {new_decision_percentage}%\n")
#   except:
#     print("-new decision percentage-- = 0% \n")
#   try:
#     No_Change_percentage = ((judgement.count('no change'))/length)*100
#     print(f"---No Change percentage--- = {No_Change_percentage}%")
#   except:
#     print("---No Change percentage--- = 0% \n")
 
#   print(f'\n Length    = {length}')
#   print(f'\n Remaining = {length-i}')
#   print('\n\n')
#   print("------------------------------------------")
#   print('\n\n')